In [1]:
import numpy as np
import math
import pandas as pd
import sklearn
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn import preprocessing #scaling
from keras.layers import Dropout            #for random dropout
from keras.optimizers import Adam   #for adam optimizer
from keras import regularizers      #for l2 regularization
from keras.wrappers.scikit_learn import KerasRegressor 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score

E:\Program Files\Anaconda3\envs\independent_study\lib\site-packages\PIL\Image.py:121: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: "9.2.0"
Pillow version: 9.2.0
  warnings.warn(str(v), RuntimeWarning)
E:\Program Files\Anaconda3\envs\independent_study\lib\site-packages\PIL\Image.py:121: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: "9.2.0"
Pillow version: 9.2.0
  warnings.warn(str(v), RuntimeWarning)


In [2]:
def bin_to_dataframe(fileName):
    data = np.fromfile(fileName, dtype='uint32')
    plain = []
    cipher = []
    for i in range(len(data)):
        if i % 2 == 0:
            plain.append(data[i])
        else:
            cipher.append(data[i])
    df_plain = pd.DataFrame(plain)
    df_plain.columns = ['plaintext']
    df_cipher = pd.DataFrame(cipher)
    df_cipher.columns = ['ciphertext']
    df_combo = pd.concat([df_plain, df_cipher], axis = 1)
    key = fileName.strip('.bin')
    key = ''.join(key.split())
    df_combo["key"] = key
    return df_combo

In [3]:
'''
    Bin file is generated from the 2x2 or 3x3 AES implementation. Currently only combines two files,
    but more can be used. Key is the file name in binary.
'''
df1 = bin_to_dataframe('01100110 01110101 01101100.bin') # key = ful
df2 = bin_to_dataframe('01101101 01100101 01100111.bin') # key = meg
combined_data = pd.concat([df1, df2])
print(combined_data)

           plaintext  ciphertext                       key
0          825241648   808464688  011001100111010101101100
1          808464689   825307184  011001100111010101101100
2          825241648   825241649  011001100111010101101100
3          825307184   808464433  011001100111010101101100
4         1666607958  2017547341  011001100111010101101100
...              ...         ...                       ...
10485755  2017547341  1482053208  011011010110010101100111
10485756  2000770125  2001028173  011011010110010101100111
10485757  1465009752  2017547341  011011010110010101100111
10485758  2017809485  1465271896  011011010110010101100111
10485759  1095058509  1094795585  011011010110010101100111

[20971520 rows x 3 columns]


In [4]:
keys = ['011001100111010101101100', '011011010110010101100111']
combined_data['key'] = combined_data['key'].replace( {'011001100111010101101100': 0, '011011010110010101100111': 1} )
target_column = ['key']
predictors = combined_data.columns.values.tolist()
predictors.remove('key')

print(combined_data[predictors].values)
print(target_column)
print(predictors)

X = combined_data[predictors].values
y = combined_data[target_column].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 40)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
count_classes = y_test.shape[1]
print(count_classes)

[[ 825241648  808464688]
 [ 808464689  825307184]
 [ 825241648  825241649]
 ...
 [1465009752 2017547341]
 [2017809485 1465271896]
 [1095058509 1094795585]]
['key']
['plaintext', 'ciphertext']


(14680064, 2)

In [9]:
'''
    model = Sequential()
    model.add(Dense(100, activation='relu', input_dim=2))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20)
    pred_train= model.predict(X_train)
    scores = model.evaluate(X_train, y_train, verbose=0)
    print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   

    pred_test= model.predict(X_test)
    scores2 = model.evaluate(X_test, y_test, verbose=0)
    print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1])) 
'''

In [7]:
n_cols = len(predictors)
input_shape = (n_cols, )
# Creates a model given an activation and learning rate
def create_model(learning_rate = 0.01, activation = 'relu'):
  
    # Create an Adam optimizer with the given learning rate
    opt = Adam(lr=learning_rate)
  
    # Create your binary classification model  
    model = Sequential()
    model.add(Dense(100, 
                    activation = activation,
                    input_shape = input_shape,
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dense(50,
                    activation = activation, 
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dense(20,
                    activation = activation, 
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dense(2, activation = activation))
    # Compile the model
    model.compile(optimizer = opt,
                  loss = "mean_absolute_error",
                  metrics=['mse', "mape"])
    return model

In [ ]:
# Create a KerasClassifier object
model = KerasRegressor(build_fn = create_model,
                       verbose = 1)
# Define the hyperparameter space
params = {'activation': ["relu", "tanh"],
          'batch_size': [16, 32, 64], 
          'epochs': [6, 9, 12],
          'learning_rate': [0.01, 0.001, 0.0001]}
# Create a randomize search cv object 
random_search = RandomizedSearchCV(model,
                                   param_distributions = params,
                                   cv = KFold(10))
random_search_results = random_search.fit(combined_data[predictors].values, combined_data[target_column].values)
print("Best Score: ",
      random_search_results.best_score_,
      "and Best Params: ",
      random_search_results.best_params_)

Epoch 1/6
294912/294912 [==============================] - 194s 659us/step - loss: 0.4285 - mse: 0.4019 - mape: 403967680.0000
Epoch 2/6
294912/294912 [==============================] - 203s 689us/step - loss: 0.4247 - mse: 0.3940 - mape: 396583776.0000
Epoch 3/6
294912/294912 [==============================] - 218s 739us/step - loss: 0.4245 - mse: 0.3920 - mape: 394448704.0000
Epoch 4/6
294912/294912 [==============================] - 374s 1ms/step - loss: 0.4243 - mse: 0.3916 - mape: 394002592.0000
Epoch 5/6
294912/294912 [==============================] - 483s 2ms/step - loss: 0.4243 - mse: 0.3910 - mape: 393161408.0000
Epoch 6/6
32768/32768 [==============================] - 35s 1ms/step - loss: 0.9918 - mse: 0.9803 - mape: 989869440.0000
Epoch 1/6
294912/294912 [==============================] - 444s 2ms/step - loss: 0.3425 - mse: 0.2859 - mape: 305292864.0000
Epoch 2/6
294912/294912 [==============================] - 211s 716us/step - loss: 0.3389 - mse: 0.2842 - mape: 304584544.

In [ ]:
'''
    Current values used (learning rate, activation function, epochs, etc.) are default
    values, should be changed after random_search_results reports the best parameters.
'''

n_cols = len(predictors)
input_shape = (n_cols, )
# Creates a model given an activation and learning rate
def create_model(learning_rate = 0.01, activation = 'relu'):
  
    # Create an Adam optimizer with the given learning rate
    opt = Adam(lr=learning_rate)
  
    # Create your binary classification model  
    model = Sequential()
    model.add(Dense(100, 
                    activation = activation,
                    input_shape = input_shape,
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dense(50,
                    activation = activation, 
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dense(20,
                    activation = activation, 
                    activity_regularizer = regularizers.l2(1e-5)))
    model.add(Dense(2, activation = activation))
    # Compile the model
    model.compile(optimizer = opt,
                  loss = "mean_absolute_error",
                  metrics=['mse', "mape"])
    return model

In [ ]:
'''
    Current values used (epochs, batch size etc.) are default
    values, should be changed after random_search_results reports the best parameters.
    CV, the cross validation value, can be changed too.
'''

# Create a KerasClassifier object
model = KerasRegressor(build_fn = create_model,
                       epochs = 6
                       batch_size = 16
                       verbose = 1)
# Calculate the accuracy score for each fold
kfolds = cross_val_score(model,
                         predictors,
                         target_column,
                         cv = 10)
# Print the mean accuracy
print('The mean accuracy was:', kfolds.mean())
# Print the accuracy standard deviation
print('With a standard deviation of:', kfolds.std())